In [ ]:
import sys, os, glob
sys.path.append('../')

import tensorflow as tf
import numpy as np

from MRIsegm.utils import get_slices
from MRIsegm.processing import denoise_slices, predict_slices, resize_slices, contour_slices
from MRIsegm.metrics import dice_coef
from MRIsegm.losses import DiceBCEloss

from PIL import Image

In [ ]:
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)

In [ ]:
models_dir = '../data/models/'
model_name = 'efficientnetb0_256_256_BTC=8_alpha3_OPT=Adam_LOSS=DiceBCEloss'
model_path = os.path.join(models_dir, model_name + '.h5')

dependencies = {
        'DiceBCEloss': DiceBCEloss,
        'dice_coef': dice_coef,
        'FixedDropout': tf.keras.layers.Dropout(0.2)
    }

model = tf.keras.models.load_model(model_path, custom_objects=dependencies)

In [ ]:
src = '/Users/giuseppefilitto/Pazienti_anonym_sorted'

patients = os.listdir(src)
if '.DS_Store' in patients:
    patients.remove('.DS_Store')
print(f"Number of patients: {len(patients)}")

In [ ]:
alpha = 3

unwanted = ['ROI', '_frames', 'bis', '_predicted', '_resized', '_countered']

for patient in patients:

    print(f'{patient}')
    try:
        folders=glob.glob(src + '/' + patient + '/T2*')
        folders = [x for x in folders if all(y not in x for y in unwanted)]
        assert len(folders) != 0
    except:
        folders=glob.glob(src + '/' + patient + '/t2*')
        folders = [x for x in folders if all(y not in x for y in unwanted)]



    for path in folders:
        folder_name = os.path.split(path)[1]
        print(f' folder: {folder_name}')
        slices_raw = get_slices(dir_path=path)
        slices = denoise_slices(slices_raw, alpha = alpha)

        predicted_ = predict_slices(slices, model, IMAGE_HEIGHT, IMAGE_WIDTH, dtype=np.uint8)
        predicted = np.squeeze(predicted_, axis=-1)


        predicted_folder = '_'.join((path, 'predicted'))
        if not os.path.exists(predicted_folder):
            os.makedirs(predicted_folder)

        for i in range(predicted.shape[0]):
            
            img = Image.fromarray(predicted[i, ...]*255)
            filename = str(i + 1) + '.png'
            output =  os.path.join(predicted_folder, filename)
            if os.path.isfile(output):
                os.remove(output)
            
            img.save(output)
    
    
        resized_ = resize_slices(slices, IMAGE_HEIGHT, IMAGE_WIDTH)
        resized = np.squeeze(resized_, axis=-1)

        resized_folder = '_'.join((path, 'resized'))
        if not os.path.exists(resized_folder):
            os.makedirs(resized_folder)

        
        for k in range(resized.shape[0]):

            img = Image.fromarray(resized[k, ...]*255).convert('L')
            filename = str(k + 1) + '.png'
            output =  os.path.join(resized_folder, filename)

            if os.path.isfile(output):
                os.remove(output)
            
            img.save(output)
            

        countured = contour_slices(resized_, predicted_)

        countured_folder = '_'.join((path, 'countured'))
        if not os.path.exists(countured_folder):
            os.makedirs(countured_folder)


        for z in range(countured.shape[0]):

            img = Image.fromarray(countured[z, ...], mode='RGB')
            filename = str(z + 1) + '.png'
            output =  os.path.join(countured_folder, filename)

            if os.path.isfile(output):
                os.remove(output)
            
            img.save(output)
    